<a href="https://colab.research.google.com/github/phmcasimiro/InLoco_Colab/blob/main/InLoco_Proj_EquipamentosSegurancaERJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FOLIUM MAP - EQUIPAMENTOS DE SEGURANÇA PÚBLICA DO RIO DE JANEIRO

## **1 - Preparação do Virtual Enviromente (VEnv)**
### **1.1 -Preparar o ambiente do COLAB para rodar as bibliotecas e pacotes**

In [1]:
# Atualizar as bibliotecas: Listar todos os pacotes instalados e quais possuem atualização
!apt update -q

# Atualizar as versões de pacotes instalados: Atualizar os pacotes listados acima para suas últimas versões
!apt upgrade -q

# Instalar a bibliteca OSM
#!pip install osmnx -q

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,545 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,403 kB]
Get:1

In [2]:
# Instalar pacotes e versões porque necessários para resolver problemas de incompatibilidade entre bibliotecas
!pip install requests -q
!pip install folium -q
!pip install imgaug -q
!pip install mplleaflet -q
!pip install geopandas -q
!pip install matplotlib -q
!pip install fiona -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 74.5 MB/s eta 0:00:00


### **1.2 - Importar as bibliotecas que serão utilizadas**

In [3]:
# Importar bibliotecas
#import osmnx as ox
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd

### **1.3 - Preparar acesso ao GDrive**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **2 - Preparar Dados e Informações para o Mapa**


*   Entradas como .shp;
*   Entradas como .gjson;
*   Criar os GDFs;
*   Verificar/Transformar os CRSs;
*   Verificar os registros iniciais dos GDFs;
*   Listar as colunas dos GDFs
*   Listar os valores únicos do atributo que será utilizado na classificação do mapa;



In [5]:
# Definir variáveis que receberão os arquivos que usaremos no projeto
aisp_shp = '/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/gisdb_seguranca_gt_seg_aisp.shp'
cisp_shp = '/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/gisdb_seguranca_gt_seg_cisp.shp'
aisp_json = '/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/aisp_erj.geojson'
cisp_json = '/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/cisp_erj.geojson'
bpm_shp = '/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/gt_seg_batalhoes_pmerj.shp'
delegacias_shp = '/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/gt_seg_delegacias_pcerj.shp'

In [6]:
# Importar variáveis/arquivos de entrada para objetos GeoDataFrame utilizando a biblioteca GeoPandas 
gdf_bpm = gpd.read_file(bpm_shp)
gdf_delegacias = gpd.read_file(delegacias_shp)
gdf_aisp = gpd.read_file(aisp_shp)
gdf_cisp = gpd.read_file(cisp_shp)

In [7]:
# Verificar os CRS
print(gdf_bpm.crs)
# Verificar o CRS
print(gdf_delegacias.crs)

EPSG:4326
EPSG:4326


In [8]:
# Exibir os registros iniciais do GeoDataFrame
gdf_bpm.head()

,unidade,endereco,municipio,telefone,telefone_2,e_mail,aisp,area_km2,globalid,created_us,created_da,last_edite,last_edi_1,geometry
0,2º BPM - Botafogo,"R. Álvaro Ramos, 135 - Botafogo, Rio de Janeir...",Rio de Janeiro,Oficial de Dia 2332-1358,NaN,NaN,0,15.024819,{B82A516E-720D-437D-AF4A-F48404678796},SEGURANCA,2022-12-28,SEGURANCA,2022-12-28,POINT (-43.18150 -22.95644)
1,PPM - Cascadura,"Av. Dom Hélder Câmara, 10199 - Cascadura, Rio ...",Rio de Janeiro,NaN,NaN,NaN,9,33.647731,{20016BA3-6552-4013-8556-4F4D2B39D2BF},NaN,NaN,NaN,NaN,POINT (-43.32560 -22.88224)
2,8º BPM - Campos,Rua Ten Cel Cardoso s/nº - Campos dos Goytacaz...,Campos dos Goytacazes,Oficial de Dia: 2731-0614,Sala de Operações: 2726-4503 / 2738-0809,secretario_8bpm@pmerj.rj.gov.br,8,6635.740494,{1BA9D17E-88A8-4606-B6ED-98B8F4B618A5},NaN,NaN,NaN,NaN,POINT (-41.31730 -21.76109)
3,15º BPM - Duque de Caxias,Rua Pedro Corrêa nº 273 – Duque de Caxias - Ce...,Duque de Caxias,Oficial de Dia 3777-7961,Sala de Operações 3777-7286,secretario_15bpm@pmerj.rj.gov.br,15,467.641196,{3F4F386E-F3D1-4918-9F8F-FAE5C74A01E0},NaN,NaN,NaN,NaN,POINT (-43.30980 -22.78110)
4,PPM - São João de Meriti,"Av. Automóvel Clube, S/N - Vilar dos Teles, Sã...",São João de Meriti,NaN,NaN,NaN,21,35.219135,{6F22237D-697D-47AF-BA28-7DFD5C8EB42C},NaN,NaN,NaN,NaN,POINT (-43.36442 -22.78667)


In [20]:
# Verificar as colunas/atributos do GeoDataFrame
gdf_bpm.columns

Index(['unidade', 'endereco', 'municipio', 'telefone', 'telefone_2', 'e_mail',
       'aisp', 'area_km2', 'globalid', 'created_us', 'created_da',
       'last_edite', 'last_edi_1', 'geometry'],
      dtype='object')

In [9]:
# Verificar os registros/valores únicos de um atributo/coluna do GeoDataFrame
pd.unique(gdf_bpm['municipio'])

array(['Rio de Janeiro', 'Campos dos Goytacazes', 'Duque de Caxias',
       'São João de Meriti', 'Cabo Frio', 'Barra do Piraí', 'Itaperuna',
       'Angra dos Reis', 'Nova Friburgo', 'São Gonçalo', 'Niterói',
       'Volta Redonda', 'Magé', 'Teresópolis', 'Resende', 'Mesquita',
       'Belford Roxo', 'Três Rios', 'Macaé', 'Santo Antônio de Pádua',
       'Petrópolis', 'Queimados', 'Itaboraí'], dtype=object)

In [10]:
# Exibir os registros iniciais do GeoDataFrame
gdf_delegacias.head()

,cisp,aisp,delegacia,localidade,categoria,endereco,responsave,contato,pip_dk_nov,pip_dscr_n,...,regiao_erj,latitude,longitude,dt_atualiz,globalid,created_us,created_da,last_edite,last_edi_1,geometry
0,118,25,118ª DP - Araruama,Interior,Interior,"Rua Bernardo Vasconcelos, 755 - Centro, Ararua...",Responsável: Delegado de Polícia Filipi Poeys ...,Telefones: (22) 2665-7886 / 2665-8403 / 2665-7758,NaN,NaN,...,Região das Baixadas Litorâneas,-22.867090,-42.337228,2022-02-22,{82391AAC-E355-42EA-952C-E4A36FB61F13},NaN,NaN,NaN,NaN,POINT (-42.33723 -22.86709)
1,118,25,NUAM (Núcleo de Atendimento à Mulher) 118ª DP ...,Interior,Atendimento à Mulher,"Rua Bernardo Vasconcelos, 755 - Centro, Ararua...",NaN,Telefones: (22) 2665-7886 / 2665-8403 / 2665-7758,NaN,NaN,...,Região das Baixadas Litorâneas,-22.867090,-42.337228,2022-02-22,{7DADA734-D716-440E-AB03-D3233E0C9A90},NaN,NaN,NaN,NaN,POINT (-42.33723 -22.86709)
2,120,35,120ª DP - Silva Jardim,Interior,Interior,"Rua Geraldo Gerônimo da Silva, 634-726 - Regin...",Responsável: Delegado de Polícia Flavio da Ros...,Telefones: (22) 2668-0434 / 2668-1891 / 2668-1894,NaN,NaN,...,Região das Baixadas Litorâneas,-22.649132,-42.389491,2022-02-22,{01DA85F1-725D-40DB-BE6D-1A8D9909BD0F},NaN,NaN,NaN,NaN,POINT (-42.38949 -22.64913)
3,0,0,ICCE - Instituto de Criminalística Carlos Ébol...,Especializada,Polícia Técnica,"Avenida Francisco Bicalho, 250, Santo Cristo, ...",Responsável: Sérgio William Silva Miana,Telefones: 2334-8476,NaN,NaN,...,Região Metropolitana,-22.905447,-43.210527,2022-02-22,{BBC85206-6E65-454F-8EB2-E9685001106D},NaN,NaN,NaN,NaN,POINT (-43.21053 -22.90545)
4,0,0,DGPAM - Departamento-Geral de Polícia de Atend...,Capital,Atendimento à Mulher,"Rua da Relação, 42 - 11º andar - Centro, Rio d...",Responsável: Delegada de Polícia Sandra Maria ...,Telefones: 2334-9749 / 2334-9814,NaN,NaN,...,Região Metropolitana,-22.910238,-43.184353,2022-02-22,{E546E0ED-6799-4BF3-AA6A-75D73D639DA0},NaN,NaN,NaN,NaN,POINT (-43.18435 -22.91024)


In [21]:
# Verificar as colunas/atributos do GeoDataFrame
gdf_delegacias.columns

Index(['cisp', 'aisp', 'delegacia', 'localidade', 'categoria', 'endereco',
       'responsave', 'contato', 'pip_dk_nov', 'pip_dscr_n', 'pip_dk_n_1',
       'pip_dscr_1', 'pip_dk_n_2', 'pip_dscr_2', 'cod_craai', 'craai',
       'cod_mun', 'municipio', 'regiao_ibg', 'mesoregiao', 'regiao_erj',
       'latitude', 'longitude', 'dt_atualiz', 'globalid', 'created_us',
       'created_da', 'last_edite', 'last_edi_1', 'geometry'],
      dtype='object')

In [11]:
# Verificar os registros/valores únicos de um atributo/coluna do GeoDataFrame
pd.unique(gdf_delegacias['regiao_erj'])

array(['Região das Baixadas Litorâneas', 'Região Metropolitana',
       'Região Noroeste Fluminense', 'Região Serrana',
       'Região Centro-Sul Fluminense', 'Região da Costa Verde',
       'Região Norte Fluminense', 'Região do Médio Paraíba'], dtype=object)

## **3 - Preparar Mapa**

*   Criar os dicionários de classificação de dados
*   Criar uma instância de Mapa Folium *folium.Map()*;
*   Criar *folium.FeatureGroup()* para agrupar geometrias de ponto e definir estilos;
*   Criar um *folium.GeoJson*() para plotar a geometria de polígono das AISPs;
*   Criar um objeto *folium.LayerControle()*;

### <font color='yellow'>Importante:</font> Todos os trechos de código ligados à publicação de camadas no mapa devem estar na mesma célula



In [12]:
cores_bpm = {'Rio de Janeiro':'black',
'Campos dos Goytacazes':'blue',
'Duque de Caxias':'green',
'São João de Meriti':'yellow',
'Cabo Frio':'orange',
'Barra do Piraí':'darkblue',
'Itaperuna':'darkgreen',
'Angra dos Reis':'cadtblue',
'Nova Friburgo':'pink',
'São Gonçalo':'lightgray',
'Niterói':'red',
'Volta Redonda':'gray',
'Magé':'brown',
'Teresópolis':'beige',
'Resende':'darkpurple',
'Mesquita':'purple',
'Belford Roxo':'lightblue',
'Três Rios':'lightgreen',
'Macaé':'yellow',
'Santo Antônio de Pádua':'black',
'Petrópolis':'white',
'Queimados':'pink',
'Itaboraí':'red'}

In [13]:
cores_delegacias={
'Região das Baixadas Litorâneas':'yellow',
'Região Metropolitana':'darkblue',
'Região Noroeste Fluminense':'gray',
'Região Serrana':'red',
'Região Centro-Sul Fluminense':'purple',
'Região da Costa Verde':'darkgreen',
'Região Norte Fluminense':'blach',
'Região do Médio Paraíba':'orange'
}

In [22]:
# Dicionário para Estilizar a camada AISP
bordersStyle = {
    'color': 'black',
    'weight': 2,
    'fillColor': 'gray',
    'fillOpacity': 0.2}

In [32]:
# Criar um mapa da folium. 
# Definir localização inicial do mapa (location), nível de zoom (zoom), escala de visibilidade (control_scale)
RMRJ_Map = folium.Map(location=[-21.776276, -43.352406], tiles = 'OpenStreetMap', zoom_start=8, control_scale=True, width='90%', height='90%')

# Criar um "folium.FeatureGroup" com um laço de repetição do tipo "for" para plotar o GeoDataFrame 'delegaciais_erj'
delegacias_group = folium.FeatureGroup('DPs')
for index, DPs in gdf_delegacias.iterrows():
  label_dp = DPs['delegacia']
  tooltip_dp = DPs['delegacia']
  folium.Marker(
      [DPs.geometry.y, DPs.geometry.x],
      icon = folium.Icon(color=cores_delegacias[DPs['regiao_erj']],
                         icon='user',
                         prefix='fa'),
      popup = folium.Popup(html= label_dp + """<img src="https://upload.wikimedia.org/wikipedia/commons/4/41/Bras%C3%A3oPCERJ.jpg" alt="Trulli" style="max-width:100%;max-height:100%">""", max_width=450),
      tooltip = tooltip_dp
      ).add_to(RMRJ_Map)
delegacias_group.add_to(RMRJ_Map)

# Criar um "folium.FeatureGroup" "bpm_group" com um laço de repetição do tipo "for" para plotar o GeoDataFrame 'bpm_erj'
bpm_group = folium.FeatureGroup('BPMs')

for index, BPMs in gdf_bpm.iterrows():

  label_bpm = BPMs['unidade']
  tooltip_bpm = BPMs['unidade']

  folium.Marker(
      [BPMs.geometry.y, BPMs.geometry.x],
      icon = folium.Icon(color=cores_bpm [BPMs['municipio']], 
                         icon='building', 
                         prefix='fa'),
      popup=folium.Popup(html = label_bpm + """<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQYQBr_I2xyMMQdeluAkD0W2KnlhHMzCH-n0Jy8eUFZXe6pWsVmHn2Gtzk1lCGj7LXsnwI&usqp=CAU" alt="Trulli" style="max-width:100%;max-height:100%">""", max_width=450),
      tooltip = tooltip_bpm
      ).add_to(RMRJ_Map)

bpm_group.add_to(RMRJ_Map)

# Exibir os limites de AISPs do ERJ
folium.GeoJson(
    data=(open(aisp_json, 'r').read()),
    tooltip=folium.GeoJsonTooltip(fields=['aisp'], aliases=['AISP']),
    name="AISPs",
    style_function=lambda x: bordersStyle
    ).add_to(RMRJ_Map)

# Adicionar controle de camadas no mapa
folium.LayerControl().add_to(RMRJ_Map)

<ipython-input-32-6034f226a311>:12: UserWarning: color argument of Icon should be one of: {'lightred', 'black', 'white', 'orange', 'blue', 'gray', 'green', 'purple', 'red', 'lightgreen', 'pink', 'lightgray', 'darkgreen', 'cadetblue', 'darkred', 'darkblue', 'darkpurple', 'beige', 'lightblue'}.
  icon = folium.Icon(color=cores_delegacias[DPs['regiao_erj']],
<ipython-input-32-6034f226a311>:30: UserWarning: color argument of Icon should be one of: {'lightred', 'black', 'white', 'orange', 'blue', 'gray', 'green', 'purple', 'red', 'lightgreen', 'pink', 'lightgray', 'darkgreen', 'cadetblue', 'darkred', 'darkblue', 'darkpurple', 'beige', 'lightblue'}.
  icon = folium.Icon(color=cores_bpm [BPMs['municipio']],


In [ ]:
# Visualizar o mapa final
RMRJ_Map

## **4 - Salvar Mapa Final**

*   Criar uma variável que armazena o caminho no qual você quer salvar o produto final; <br> <font color='yellow'>(lembrar de colocar um 'r' antes do caminho e uma barra após a última pasta)</font>
*   Criar uma variável para armazenar o nome de salvamento do arquivo;
*   Utilizar a função *.save()* para salvar o arquivo 

In [36]:
# Salvar o mapa em formato .html como arquivo local
# Caminho/Diretório para salvar o arquivo
#outfp=r'/content/drive/MyDrive/11_PROJETOS/FogoCruzado_RMRJ/'

# Defina o caminho para salvar o mapa
#fp = "EquipamentosSegurancaERJ.html"
# Exporte o mapa
#RMRJ_Map.save(outfp+fp)